In [116]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score, auc, precision_recall_curve
import math
import scipy
from scipy.optimize import minimize
from scipy.spatial import ConvexHull
from scipy.linalg import norm
from scipy.optimize import minimize_scalar
import datetime
import time
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [117]:
dfs = scipy.io.loadmat("/content/benchmarks.mat")
#crate thyroid dataframe
x_1 = dfs['thyroid']['x'][0, 0]
t_1 = dfs['thyroid']['t'][0, 0]

# Combina x e t in un array con 4 colonne
combined_data = np.hstack((x_1, t_1))

# Crea il DataFrame
column_names = ['Column' + str(i) for i in range(1, 7)]
data_1 = pd.DataFrame(combined_data, columns=column_names)
data_1.rename(columns={'Column6': 'Class'}, inplace=True)

#create titanic dataframe
x_2 = dfs['heart']['x'][0, 0]
t_2 = dfs['heart']['t'][0, 0]

# Combina x e t in un array con 4 colonne
combined_data = np.hstack((x_2, t_2))

# Crea il DataFrame
column_names = ['Column' + str(i) for i in range(1, 15)]
data_2 = pd.DataFrame(combined_data, columns=column_names)
data_2.rename(columns={'Column14': 'Class'}, inplace=True)

In [118]:
data=data_1

In [119]:
data

,Column1,Column2,Column3,Column4,Column5,Class
0,-3.846984,1.796980,7.350487,-0.226893,-0.496052,1.0
1,-3.846984,3.330645,3.538182,-0.226893,-0.483855,1.0
2,-3.673618,2.898627,4.990489,-0.153984,-0.581431,1.0
3,-3.586935,1.040948,5.353566,-0.328966,-0.532643,1.0
4,-2.893473,3.330645,2.358183,-0.241475,-0.520446,1.0
...,...,...,...,...,...,...
210,2.567545,-1.227148,-1.091045,0.210561,0.260162,1.0
211,2.567545,1.408164,1.722799,-0.256057,-0.532643,1.0
212,2.740911,-1.594364,-0.546430,0.822998,0.406526,1.0
213,2.740911,-0.924735,-0.092585,0.925071,1.248120,1.0


In [120]:
def weak_learner(X, y, w):
    clf = DecisionTreeClassifier(max_depth=2, random_state=np.random.randint(100))
    clf.fit(X, y,sample_weight=w)
    return clf

In [121]:
def encode_hypothesis(h, y_train):
  y_train_np = y_train.to_numpy()
  e_j = []
  for h_i, y_i in zip(h, y_train_np):
      if h_i != y_i:
          e_j.append(1)
      else:
          e_j.append(0)
  return e_j

In [122]:
def delta(d):
    m = len(d)
    x = np.log(d)
    y = np.log(m)
    return np.sum(d.T * np.log(d)) + np.log(m)

def expression(d):
    A_array = A['Class'].values
    Aw = np.dot(A_array, w)
    return np.sum(d.T * Aw) + (1/N) * delta(d)

def minimize_expression(A, w_1, N, v):
    m = A.shape[0]  # Numero di righe nella matrice A
    constraint = ({'type': 'eq', 'fun': lambda d: np.sum(d) - 1})
    d = np.random.rand(m) / v  # Generazione casuale di d in [0, 1/v]^m (ad esempio)
    d = d/ np.sum(d)
    result = minimize(lambda d: expression(d), d, constraints=constraint)
    return d, result.x

In [123]:
def f(w):
    A_array = A['Class'].values
    return np.sum(np.dot(d.T, np.dot(A_array, w)))
def update_weights(f_w):
    min_value = float('inf')
    minimizing_vector = None
    for vector in f_w:
        result = minimize(lambda x: f(x), vector)
        value = result.fun
        if value < min_value:
            min_value = value
            minimizing_vector = vector
    return minimizing_vector

In [124]:
def secondary_weight(A, d, w):
    A_array = A['Class'].values
    return np.sum(np.dot(A_array, w) * d)

def minimize_secondary_weight(A, w):
    m = A.shape[0]  # Numero di righe nella matrice A
    constraint = ({'type': 'eq', 'fun': lambda d: np.sum(d) - 1})
    d = np.random.rand(m) / v  # Generazione casuale di d in [0, 1/v]^m (ad esempio)
    d = d /np.sum(d)
    result = minimize(lambda d: secondary_weight(A, d, w), d, constraints=constraint)
    return result.fun

def maximize_w_in_convex_hull(E_t_plus_1, A, w):
    max_value = float('-inf')
    max_w = None
    for w_candidate in E_t_plus_1:
      value = minimize_secondary_weight(A, w_candidate)
      if value > max_value:
          max_value = value
          max_w = w_candidate
    return max_w

In [125]:
def short_step_FW(A, ej2, w, result):
    lamb = []
    ej2_np = np.array(ej2)
    w_np = np.array(w)
    val = (ej2_np - w_np)
    A_array = A['Class'].values
    numerator = result.dot(A_array) * val
    denominator = np.linalg.norm(A_array * val, ord=np.inf) ** 2
    fraction = numerator / denominator
    lambda_t = np.clip(fraction, 0, 1)
    wt1 = w + lambda_t * d
    return wt1

In [126]:
def pairwise_FW(A, d, E_t_plus_1):
    best_value = float('inf')
    best_e = None
    E_t = E_t_plus_1[:-1]
    for e in E_t_plus_1:
        value = minimize(lambda e: expression_f(A, d, e), e,)
        value_1 = value.x
        value_2 = value.fun
        if value_2 < best_value:
          best_value = value_2
          best_e = e
    return best_e
def expression_f(A, d, e):
    A_array = A['Class'].values
    return np.sum(np.dot(d.T, np.dot(A_array, e)))
def step_size(s, e_way, ej2, w, A):
    A_array = A['Class'].values
    ej2_np = np.array(ej2)
    e_away_np = np.array(e_away)
    value_0 = (ej2_np - e_away_np)
    value_1 = s * (value_0)
    value_2 = w + value_1
    value_3 = np.dot(-A_array, value_2)
    value = np.dot(d.T, np.dot(A_array, w))*value_3
    return value

In [127]:

def search_direction(w_s, A):
  index = None
  best_w_s = None
  min_value = float('inf')
  s = A.shape[1]
  for c in w_s:
      value = gradient_f(c, A)
      if value < min_value:
          min_value = value
          best_w_s = c
  return best_w_s

def gradient_f(d, A):
  A_array = A['Class'].values
  return -(np.dot(d.T, A_array))

In [128]:
def minimize_expr(A, w):
    m = A.shape[0]  # Numero di righe nella matrice A
    constraint = ({'type': 'eq', 'fun': lambda d: np.sum(d) - 1})
    d = np.random.rand(m) / v  # Generazione casuale di d in [0, 1/v]^m (ad esempio)
    d = d/ np.sum(d)
    result = minimize(lambda d: f_star(A, d, w), d, constraints=constraint)
    return result.fun
def f_star(A, d, w):
    A_array = A['Class'].values
    return  -np.min(np.dot(A_array, w) * d)

In [129]:
#Normalizing the variables to values in [-1,1]
for col in data.columns:
    if col != 'Class':
        min_val = data[col].min()
        max_val = data[col].max()
        data[col] = (data[col] - min_val) / (max_val - min_val)  # Normalize to range 0-1
        data[col] = data[col] * 2 - 1  # Rescale to range -1 to 1

In [130]:
X = data.drop('Class', axis=1)
y = data['Class']

In [131]:
num_folds = 5
kf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)
accuracy_scores = []
roc_auc_scores = []
max_accuracy_test = float('-inf')
max_accuracy_train = float('-inf')
start_time = time.time()
for train_idx, test_idx in kf.split(X, y):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    weak_classifier = weak_learner(X_train, y_train, 1)
    hj_0 = weak_classifier.predict(X_train)
    A = hj_0 * y_train
    A = pd.DataFrame(A)
    m = X_train.shape[0]
    n = len(hj_0)
    d0 = np.zeros(m)
    for i in range(m):
      d0[i] = 1/m
    #sendo d0 to the weak learner to obtain hj_1
    h= weak_learner(X_train,y_train, d0)
    hj_1 = h.predict(X_train)
    ej1 = encode_hypothesis(hj_1, y_train)
    w = np.zeros(m,)
    w=ej1
    #short step
    #set parameters
    v = 848
    T = 50
    e = 0.01
    epsilon_0 = float('inf')
    N = (2 * np.log(m/v))/e
    #create all the variables that we need
    E_t_plus_1 = [ej1]
    d_values =[]
    HT_train = np.zeros(len(X_train))
    HT_test = np.zeros(len(X_test))
    max_limit = 1
    min_limit = -1
    for t in range(1, T):
        #create the distribution d_t
        d, result = minimize_expression(A, w, N, v)
        #obtain the hypothesis h_j+1 and e_j+1
        d_values.append(result)
        h2 = weak_learner(X_train,y_train, result)
        hj_2_train = h2.predict(X_train)
        hj_2_test =h2.predict(X_test)
        ej2 = encode_hypothesis(hj_2_train, y_train)
        for tau in range(t):
            val_1 = np.min(np.dot(d_values[tau], A))
            if val_1 < epsilon_0:
                epsilon_0 = val_1
        val_2 = minimize_expr(A, w)
        epsilon = np.abs(val_2 + epsilon_0)
        if epsilon <= (e/2):
            print("achieve the opptimality gap")
            break
        E_t_plus_1.insert(t, ej2)
        #compute the FW weight
        w_1 = short_step_FW(A, ej2, w, result)
        #calculate secondary weigths
        result_w = maximize_w_in_convex_hull(E_t_plus_1, A, w)
        w_2 = result_w
        f_w = []
        f_w.insert(0, w_1)
        f_w.insert(1, w_2)
        best_w = update_weights(f_w)
        w = best_w
        HT_train += hj_2_train
        HT_train[HT_train > max_limit] = max_limit
        HT_train[HT_train < min_limit] = min_limit
        HT_test += hj_2_test
        HT_test[HT_test > max_limit] = max_limit
        HT_test[HT_test < min_limit] = min_limit
        accuracy_test = accuracy_score(HT_test, y_test)
    final_predictions_train = np.sign(HT_train)
    final_predictions_test = np.sign(HT_test)
    # Evaluate performance on the training set and test set
    accuracy_train = accuracy_score(final_predictions_train, y_train)
    accuracy_test = accuracy_score(final_predictions_test, y_test)
    if accuracy_test > max_accuracy_test:
      max_accuracy_test = accuracy_test
    if accuracy_train > max_accuracy_train:
      max_accuracy_train = accuracy_train
    roc_auc = roc_auc_score(y_test, final_predictions_test)
    precision, recall, _ = precision_recall_curve(y_test, final_predictions_test)
    pr_auc = auc(recall, precision)
    accuracy_scores.append(accuracy_test)
    roc_auc_scores.append(roc_auc)
# mean accuracy and mean AUC-ROC
end_time = time.time()
overall_time = end_time - start_time
print("overall time is {}".format(overall_time))
mean_accuracy = np.mean(accuracy_scores)
mean_roc_auc = np.mean(roc_auc_scores)

# std accuracy ans std AUC-ROC
std_accuracy = np.std(accuracy_scores)
std_roc_auc = np.std(roc_auc_scores)

#  print results
print(f"Mean Accuracy: {mean_accuracy:.2f} +/- {std_accuracy:.2f}")
print(f"Mean AUC-ROC: {mean_roc_auc:.4f} +/- {std_roc_auc:.4f}")
print("Max accuracy in the train set is {}".format(max_accuracy_train))
print("Max accuracy in the test set is {}".format(max_accuracy_test))

overall time is 568.5634331703186
Mean Accuracy: 0.90 +/- 0.04
Mean AUC-ROC: 0.8959 +/- 0.0436
Max accuracy in the train set is 0.9534883720930233
Max accuracy in the test set is 0.9534883720930233


In [132]:
num_folds = 5
kf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)
accuracy_scores = []
roc_auc_scores = []
max_accuracy_test = float('-inf')
max_accuracy_train = float('-inf')
start_time = time.time()
for train_idx, test_idx in kf.split(X, y):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    weak_classifier = weak_learner(X_train, y_train, 1)
    hj_0 = weak_classifier.predict(X_train)
    A = hj_0 * y_train
    A = pd.DataFrame(A)
    m = X_train.shape[0]
    n = len(hj_0)
    d0 = np.zeros(m)
    for i in range(m):
      d0[i] = 1/m
    #sendo d0 to the weak learner to obtain hj_1
    h= weak_learner(X_train,y_train, d0)
    hj_1 = h.predict(X_train)
    ej1 = encode_hypothesis(hj_1, y_train)
    w = np.zeros(m,)
    w=ej1
    #away step
    #set parameters
    v = 848
    T = 50
    e = 0.01
    epsilon_0 = float('inf')
    N = (2 * np.log(m/v))/e
    #create all the variables that we need
    E_t_plus_1 = [ej1]
    d_values =[]
    HT_train = np.zeros(len(X_train))
    HT_test = np.zeros(len(X_test))
    max_limit = 1
    min_limit = -1
    for t in range(1, T):
        #create the distribution d_t
        d, result = minimize_expression(A, w, N, v)
        #obtain the hypothesis h_j+1 and e_j+1
        if np.all(np.isnan(result)):
          z = 1e-10
          result.fill(z)
        d_values.append(result)
        h2 = weak_learner(X_train,y_train, result)
        hj_2_train = h2.predict(X_train)
        hj_2_test = h2.predict(X_test)
        ej2 = encode_hypothesis(hj_2_train, y_train)
        for tau in range(t):
            val_1 = np.min(np.dot(d_values[tau], A))
            if val_1 < epsilon_0:
                epsilon_0 = val_1
        val_2 = minimize_expr(A, w)
        epsilon = np.abs(val_2 + epsilon_0)
        if epsilon <= (e/2):
            print("achieve the opptimality gap")
            break
        E_t_plus_1.insert(t, ej2)
        #compute the Away-step FW weight
        e_away = pairwise_FW(A, result, E_t_plus_1)
        a = []
        for i in range(len(e_away)):
          if e_away[i] == w[i]:
            a.append(1)
          else:
            a.append(0)
        a_index = np.argmax(a)
        w_s = []
        ej2_np = np.array(ej2)
        w_np = np.array(w)
        w_s.insert(0, (ej2_np - w_np))
        w_s.insert(1, (w_np - e_away))
        w_d = search_direction(w_s, A)
        if np.array_equal(w_d, w_s[0]):
          w_1_1 = w_d
          lamb_t_max = 1
          best_lamb = float('inf')
        else:
          w_1_1 = w_d
          if (1 - a[a_index]) == 0 :
            lamb_t_max = 0
            best_lamb = 0
          else:
            lamb_t_max = a[a_index] / (1 - a[a_index])
            best_lamb = float('inf')
        #calculate lamb_t
        for s in range(lamb_t_max):
            value_0 = step_size(s, e_away, ej2, w, A)
            lamb_index = np.argmin(value_0)
            value = value_0[lamb_index]
            if value < best_lamb:
                best_lamb = value
        w_1 = w + best_lamb * w_1_1
        #calculate secondary weigths
        result_w = maximize_w_in_convex_hull(E_t_plus_1, A, w)
        w_2 = result_w
        f_w = []
        f_w.insert(0, w_1)
        f_w.insert(1, w_2)
        best_w = update_weights(f_w)
        w = best_w
        HT_train += hj_2_train
        HT_train[HT_train > max_limit] = max_limit
        HT_train[HT_train < min_limit] = min_limit
        HT_test += hj_2_test
        HT_test[HT_test > max_limit] = max_limit
        HT_test[HT_test < min_limit] = min_limit
    final_predictions_train = np.sign(HT_train)
    final_predictions_test = np.sign(HT_test)

    # Evaluate performance on the training set and test set (accuracy)
    accuracy_train = accuracy_score(final_predictions_train, y_train)
    accuracy_test = accuracy_score(final_predictions_test, y_test)
    roc_auc = roc_auc_score(y_test, final_predictions_test)
    precision, recall, _ = precision_recall_curve(y_test, final_predictions_test)
    pr_auc = auc(recall, precision)
    if accuracy_test > max_accuracy_test:
      max_accuracy_test = accuracy_test
    if accuracy_train > max_accuracy_train:
      max_accuracy_train = accuracy_train
    accuracy_scores.append(accuracy_test)
    roc_auc_scores.append(roc_auc)
# mean accuracy and mean AUC-ROC
mean_accuracy = np.mean(accuracy_scores)
mean_roc_auc = np.mean(roc_auc_scores)

# std accuracy ans std AUC-ROC
std_accuracy = np.std(accuracy_scores)
std_roc_auc = np.std(roc_auc_scores)
end_time = time.time()
overall_time = end_time - start_time
print("overall time is {}".format(overall_time))
# print results
print(f"Mean Accuracy: {mean_accuracy:.2f} +/- {std_accuracy:.2f}")
print(f"Mean AUC-ROC: {mean_roc_auc:.4f} +/- {std_roc_auc:.4f}")
print("Max accuracy in the train set is {}".format(max_accuracy_train))
print("Max accuracy in the test set is {}".format(max_accuracy_test))

overall time is 1546.4310383796692
Mean Accuracy: 0.86 +/- 0.04
Mean AUC-ROC: 0.8974 +/- 0.0444
Max accuracy in the train set is 0.9302325581395349
Max accuracy in the test set is 0.9069767441860465


In [133]:
num_folds = 5
kf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)
accuracy_scores = []
roc_auc_scores = []
max_accuracy_test = float('-inf')
max_accuracy_train = float('-inf')
start_time = time.time()
for train_idx, test_idx in kf.split(X, y):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    weak_classifier = weak_learner(X_train, y_train, 1)
    hj_0 = weak_classifier.predict(X_train)
    A = hj_0 * y_train
    A = pd.DataFrame(A)
    m = X_train.shape[0]
    n = len(hj_0)
    d0 = np.zeros(m)
    for i in range(m):
      d0[i] = 1/m
    #send d0 to the weak learner to obtain hj_1
    h= weak_learner(X_train,y_train, d0)
    hj_1 = h.predict(X_train)
    ej1 = encode_hypothesis(hj_1, y_train)
    w = np.zeros(m,)
    w=ej1
    #pairwise FW
    #set parameters
    v = 848
    T = 50
    e = 0.01
    epsilon_0 = float('inf')
    N = (2 * np.log(m/v))/e
    #create all the variables that we need
    E_t_plus_1 = [ej1]
    d_values =[]
    HT_train = np.zeros(len(X_train))
    HT_test = np.zeros(len(X_test))
    max_limit = 1
    min_limit = -1
    for t in range(1, T):
        #create the distribution d_t
        d, result = minimize_expression(A, w, N, v)
        #obtain the hypothesis h_j+1 and e_j+1
        if np.all(np.isnan(result)):
          z = 1e-10
          result.fill(z)
        d_values.append(result)
        h2 = weak_learner(X_train,y_train, result)
        hj_2 = h2.predict(X_train)
        ej2 = encode_hypothesis(hj_2, y_train)
        for tau in range(t):
            val_1 = np.min(np.dot(d_values[tau], A))
            if val_1 < epsilon_0:
                epsilon_0 = val_1
        val_2 = minimize_expr(A, w)
        epsilon = np.abs(val_2 + epsilon_0)
        if epsilon <= (e/2):
            print("achieve the opptimality gap")
            break
        E_t_plus_1.insert(t, ej2)
        #compute the Pairwise FW weight
        e_away = pairwise_FW(A, result, E_t_plus_1)
        a = []
        for i in range(len(e_away)):
          if e_away[i] == w[i]:
            a.append(1)
          else:
            a.append(0)
        #set lamb_t_max
        a_index = np.argmax(a)
        lamb_t_max = a[a_index]
        #calculate lamb_t
        best_lamb = float('inf')
        for s in range(lamb_t_max):
            value_0 = step_size(s, e_away, ej2, w, A)
            lamb_index = np.argmin(value_0)
            value = value_0[lamb_index]
            if value < best_lamb:
                best_lamb = value
        ej2_np = np.array(ej2)
        e_away_np = np.array(e_away)
        w_1 = w + best_lamb * (ej2_np - e_away_np)
        #calculate secondary weigths
        result_w = maximize_w_in_convex_hull(E_t_plus_1, A, w)
        w_2 = result_w
        f_w = []
        f_w.insert(0, w_1)
        f_w.insert(1, w_2)
        best_w = update_weights(f_w)
        w = best_w
        HT_train += hj_2_train
        HT_train[HT_train > max_limit] = max_limit
        HT_train[HT_train < min_limit] = min_limit
        HT_test += hj_2_test
        HT_test[HT_test > max_limit] = max_limit
        HT_test[HT_test < min_limit] = min_limit
    final_predictions_train = np.sign(HT_train)
    final_predictions_test = np.sign(HT_test)

    # Evaluate performance on the training set and test set (accuracy)
    accuracy_train = accuracy_score(final_predictions_train, y_train)
    accuracy_test = accuracy_score(final_predictions_test, y_test)
    roc_auc = roc_auc_score(y_test, final_predictions_test)
    precision, recall, _ = precision_recall_curve(y_test, final_predictions_test)
    pr_auc = auc(recall, precision)
    if accuracy_test > max_accuracy_test:
      max_accuracy_test = accuracy_test
    if accuracy_train > max_accuracy_train:
      max_accuracy_train = accuracy_train
    accuracy_scores.append(accuracy_test)
    roc_auc_scores.append(roc_auc)
# mean accuracy and mean AUC-ROC
mean_accuracy = np.mean(accuracy_scores)
mean_roc_auc = np.mean(roc_auc_scores)

# std accuracy ans std AUC-ROC
std_accuracy = np.std(accuracy_scores)
std_roc_auc = np.std(roc_auc_scores)

#time
end_time = time.time()
overall_time = end_time - start_time
print("overall time is {}".format(overall_time))
# print results
print(f"Mean Accuracy: {mean_accuracy:.2f} +/- {std_accuracy:.2f}")
print(f"Mean AUC-ROC: {mean_roc_auc:.4f} +/- {std_roc_auc:.4f}")
print("Max accuracy in the train set is {}".format(max_accuracy_train))
print("Max accuracy in the test set is {}".format(max_accuracy_test))

overall time is 1597.2306199073792
Mean Accuracy: 0.74 +/- 0.05
Mean AUC-ROC: 0.6423 +/- 0.0604
Max accuracy in the train set is 0.9302325581395349
Max accuracy in the test set is 0.8372093023255814
